In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def pad_and_concat(sequences):  # sequences shape: [batch_size, len, dims...] -> ([batch_size, maxlen, dims...], [len])
    arrays = [np.asarray(seq) for seq in sequences]
    lengths = np.asarray([array.shape[0] for array in arrays], dtype=np.int32)
    maxlen = np.max(lengths)
    arrays = [np.pad(array, [(0, maxlen - array.shape[0]), (0, 0)], 'constant', constant_values=0) for array in arrays]
    return np.asarray(arrays), lengths
    

In [3]:
small_cnf, small_lengths = pad_and_concat(
    [[[1, -2], [2, 1]],
     [[-2, -1], [1, -2]],
     [[-1, -1], [-2, -2], [-1, -2]],
      [[1, 1], [-1, -1]],
      [[1, -1], [1, 2], [-1, -2]],
      [[-2, -2], [2, 2]]
     ])
small_sol = np.asarray([1, 1, 1, 0, 0, 0])

In [4]:
print(small_cnf)
print(small_lengths)

[[[ 1 -2]
  [ 2  1]
  [ 0  0]]

 [[-2 -1]
  [ 1 -2]
  [ 0  0]]

 [[-1 -1]
  [-2 -2]
  [-1 -2]]

 [[ 1  1]
  [-1 -1]
  [ 0  0]]

 [[ 1 -1]
  [ 1  2]
  [-1 -2]]

 [[-2 -2]
  [ 2  2]
  [ 0  0]]]
[2 2 3 2 3 2]


In [5]:
VARIABLE_NUM = 3
EMBEDDING_SIZE = 8
CLAUSE_SIZE = 2
LSTM_STATE_SIZE = 8
batch_size = 6

In [6]:
def assert_shape(matrix, shape: list):
    act_shape = matrix.get_shape().as_list()
    assert act_shape == shape, "got shape {}, expected {}".format(act_shape, shape)

In [7]:
class Graph:
    def __init__(self):
        self.inputs = tf.placeholder(tf.int32, shape=(batch_size, None, CLAUSE_SIZE), name='inputs')
        self.lengths = tf.placeholder(tf.int32, shape=(batch_size,), name='lengths')
        self.labels = tf.placeholder(tf.float32, shape=(batch_size,), name='labels')
        
        vars_ = tf.abs(self.inputs)
        signs = tf.cast(tf.sign(self.inputs), tf.float32)  # shape: [batch_size, None, CLAUSE_SIZE]

        embeddings = tf.Variable(tf.random_uniform([VARIABLE_NUM + 1, EMBEDDING_SIZE], -1., 1), name='embeddings')

        var_embeddings = tf.nn.embedding_lookup(embeddings, vars_)
        # var_embeddings shape: [None, None, CLAUSE_SIZE, EMBEDDING_SIZE]
        
        clause_preembeddings = tf.concat(
            [tf.reshape(var_embeddings, [batch_size, -1, CLAUSE_SIZE * EMBEDDING_SIZE]), 
             signs],
            axis=2)
        
        PREEMBEDDING_SIZE = EMBEDDING_SIZE * CLAUSE_SIZE + CLAUSE_SIZE
        assert_shape(clause_preembeddings, 
                     [batch_size, None, PREEMBEDDING_SIZE])
        
        clause_w = tf.Variable(tf.random_normal(
            [PREEMBEDDING_SIZE, EMBEDDING_SIZE]), name='clause_w')
        clause_b = tf.Variable(tf.random_normal([EMBEDDING_SIZE]), name='clause_b')
        clause_embeddings = tf.reshape(tf.sigmoid(
            tf.reshape(clause_preembeddings, [-1, PREEMBEDDING_SIZE]) @ clause_w + clause_b), 
                                       [batch_size, -1, EMBEDDING_SIZE])
        # shape: [None, None, EMBEDDING_SIZE]
        
        lstm = tf.contrib.rnn.BasicLSTMCell(LSTM_STATE_SIZE)
        hidden_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        current_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        state = hidden_state, current_state
        
        _, lstm_final_state = tf.nn.dynamic_rnn(lstm, clause_embeddings, dtype=tf.float32, 
                                               sequence_length=self.lengths
                                               )
        formula_embedding = lstm_final_state.h
            
        assert_shape(formula_embedding, [batch_size, LSTM_STATE_SIZE])
            
        softmax_w = tf.Variable(tf.random_normal([LSTM_STATE_SIZE, 1]), name='softmax_w')
        softmax_b = tf.Variable(tf.random_normal([1]), name='softmax_b')
        
        self.logits = tf.squeeze(formula_embedding @ softmax_w, axis=1) + softmax_b
        self.loss = tf.losses.sigmoid_cross_entropy(self.labels, self.logits) 
        self.probabilities = tf.sigmoid(self.logits)

tf.reset_default_graph()
model = Graph()

In [8]:
np.set_printoptions(precision=2, suppress=True)

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    probs = sess.run([model.probabilities], feed_dict={
        model.inputs: small_cnf,
        model.lengths: small_lengths
    })
    
print(probs)

[array([0.78, 0.77, 0.78, 0.8 , 0.76, 0.79], dtype=float32)]


In [10]:
with tf.Session() as sess:
    train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(model.loss)
    sess.run(tf.global_variables_initializer())
    for _ in range(100):
        sess.run(train_op, feed_dict={
            model.inputs: small_cnf,
            model.labels: small_sol,
            model.lengths: small_lengths
        })
    probs = sess.run(model.probabilities, feed_dict={
        model.inputs: small_cnf,
        model.lengths: small_lengths
    })
    
probs

array([1., 1., 1., 0., 0., 0.], dtype=float32)

In [11]:
import cnf

In [20]:
def sat_array():
    for k in range(1, 6):
        for var_num in range(2, 5):
            for clause_num in range(3, 10):
                sat = {True: 0, False: 0}
                for _ in range(100):
                    sat[cnf.get_random_kcnf(k, var_num, clause_num).satisfiable()] += 1
                print(k, var_num, clause_num, sat)
#sat_array()

In [29]:
from sklearn.utils import shuffle
def generate_balanced_set(max_num):
    sat = {True: [], False: []}
    for formula in cnf.get_random_kcnfs(1000, 2, 3, 10):
        sat[formula.satisfiable()].append(formula)
        
    minlen = min(len(sat[True]), len(sat[False]))
    del sat[True][minlen:]
    del sat[False][minlen:]
    cnfs = sat[True] + sat[False]
    labels = [True] * minlen + [False] * minlen
    shuffle(cnfs, labels)
    return cnfs, labels

In [31]:
from sklearn.model_selection import train_test_split
train_cnfs, test_cnfs, train_labels, test_labels = train_test_split(*generate_balanced_set(1000), test_size=0.1)

In [32]:
print("train sat:", sum(1 for label in train_labels if label == True), "train unsat:", sum(1 for label in train_labels if label == False))
print("test sat:", sum(1 for label in test_labels if label == True), "train unsat:", sum(1 for label in test_labels if label == False))

train sat: 214 train unsat: 214
test sat: 24 train unsat: 24


In [16]:
def chunks(lists, chunk_size):
    return [[it[i:i + chunk_size] for it in lists] for i in range(0, len(lists[0]), chunk_size)]

In [17]:
chunks([list(range(10)), list(range(10, 20))], 2)

[[[0, 1], [10, 11]],
 [[2, 3], [12, 13]],
 [[4, 5], [14, 15]],
 [[6, 7], [16, 17]],
 [[8, 9], [18, 19]]]

In [38]:
with tf.Session() as sess:
    train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(model.loss)
    sess.run(tf.global_variables_initializer())
    
    for epoch_num in range(100):
        print("Epoch", epoch_num)
        losses = []
        accs = []
        for (batch_cnfs, batch_labels) in chunks((train_cnfs, train_labels), batch_size):
            if len(batch_cnfs) < batch_size:
                print("skipping incomplete batch")
                continue
            inputs, lengths = pad_and_concat([cnf.clauses for cnf in batch_cnfs])
            _, loss, probs = sess.run([train_op, model.loss, model.probabilities], feed_dict={
                model.inputs: inputs,
                model.labels: batch_labels,
                model.lengths: lengths
            })
            losses.append(loss)
            accs.append(1 - np.mean(np.abs(np.around(probs) - np.asarray(batch_labels))))
            
        print("train loss:", np.mean(np.asarray(losses)), "train acc:", np.mean(accs))
        
        losses = []
        accs = []
        for (batch_cnfs, batch_labels) in chunks((test_cnfs, test_labels), batch_size):
            if len(batch_cnfs) < batch_size:
                print("skipping incomplete batch")
                continue
            inputs, lengths = pad_and_concat([cnf.clauses for cnf in batch_cnfs])
            loss, probs = sess.run([model.loss, model.probabilities], feed_dict={
                model.inputs: inputs,
                model.lengths: lengths,
                model.labels: batch_labels
            })
            losses.append(loss)
            accs.append(1 - np.mean(np.abs(np.around(probs) - np.asarray(batch_labels))))
        print('test loss:', np.mean(np.asarray(losses)), 'test acc:', np.mean(accs))


    print("first batch:")
    first_inputs, first_lengths = pad_and_concat([cnf.clauses for cnf in cnfs[:batch_size]])
    probs = sess.run(model.probabilities, feed_dict={
        model.inputs: first_inputs,
        model.lengths: first_lengths
    })
    print("actual:", probs)
    print("expected", labels[:batch_size])
        
    print("new data:")
    probs = sess.run(model.probabilities, feed_dict={
        model.inputs: small_cnf,
        model.lengths: small_lengths
    })
    print("actual", probs)
    print("expected", small_sol)


Epoch 0
skipping incomplete batch
train loss: 0.6092559 train acc: 0.685446002953489
test loss: 0.5293864 test acc: 0.7291666641831398
Epoch 1
skipping incomplete batch
train loss: 0.5444753 train acc: 0.723004688469457
test loss: 0.49674332 test acc: 0.75
Epoch 2
skipping incomplete batch
train loss: 0.5109746 train acc: 0.7535211213038001
test loss: 0.4832041 test acc: 0.75
Epoch 3
skipping incomplete batch
train loss: 0.49179855 train acc: 0.7488262873720115
test loss: 0.4761938 test acc: 0.729166666045785
Epoch 4
skipping incomplete batch
train loss: 0.4722482 train acc: 0.7793427185273506
test loss: 0.4629519 test acc: 0.7499999981373549
Epoch 5
skipping incomplete batch
train loss: 0.46139812 train acc: 0.7723004650062238
test loss: 0.45140305 test acc: 0.7499999981373549
Epoch 6
skipping incomplete batch
train loss: 0.46384522 train acc: 0.7699530475156408
test loss: 0.4511507 test acc: 0.8124999944120646
Epoch 7
skipping incomplete batch
train loss: 0.4459896 train acc: 0.77934